In [27]:
import os
import sys

import torch
from torch import nn, optim
from torch.nn import functional as FeatureAlphaDropout
import torch.nn as nn
sys.path.append(os.path.realpath('../'))

import pickle as pkl
from datasets import make_dataloader
from bitrap.modeling import make_model
from bitrap.engine import build_engine

from bitrap.utils.logger import Logger
import logging
import argparse
from configs import cfg
from termcolor import colored 

from torch.utils.data import DataLoader


In [15]:
cfg.merge_from_file("../configs/bitrap_np_JAAD.yml")
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

In [17]:
model = make_model(cfg)
model = model.to(cfg.DEVICE)
checkpoint = "../" + cfg.CKPT_DIR + "best.pth"

if os.path.isfile(checkpoint):
    model.load_state_dict(torch.load(checkpoint))
    print(colored('Loaded checkpoint:{}'.format(checkpoint), 'blue', 'on_green'))
else:
    print(colored('The cfg.CKPT_DIR id not a file: {}'.format(checkpoint), 'green', 'on_red'))

Loaded checkpoint:../checkpoints/JAAD_checkpoints/goal_cvae_checkpoints/best.pth


In [18]:
_, _, inference = build_engine(cfg)

In [29]:
images = [f"../data/JAAD/images/video_0005/000{i + 46}.png" for i in range(16)]



dataloader = DataLoader(images, num_workers=1)

In [32]:
def predict(model, input_x, cur_pos = None):
    gt_goal = None
    cur_pos = input_x[:, -1, :] if cur_pos is None else cur_pos
    batch_size, seg_len, _ = input_x.shape

    h_x = model.encoder(images, None)
    Z, KLD = model.gaussian_latent_net(h_x, input_x[:, -1, :], None, z_mode=False)
    enc_h_and_z = torch.cat([h_x.unsqueeze(1).repeat(1, Z.shape[1], 1), Z], dim=-1)
    pred_goal = model.goal_decoder(enc_h_and_z)
    # dec_h = enc_h_and_z if model.cfg.DEC_WITH_Z else h_x

    pred_goal = model.goal_decoder(enc_h_and_z)

    # dec_h = enc_h_and_z if model.cfg.DEC_WITH_Z else h_x
    pred_traj = model.pred_future_traj(h_x, pred_goal)
    cur_pos = input_x[:, None, -1, :] if cur_pos is None else cur_pos.unsqueeze(1)
    pred_goal = pred_goal + cur_pos
    pred_traj = pred_traj + cur_pos.unsqueeze(1)

    return pred_traj, pred_goal



In [33]:
model.eval()

input_x = images

pred_traj, pred_goal = predict(model, input_x, cur_pos=input_x[:, -1, :cfg.MODEL.DEC_OUTPUT_DIM])




# pred_goal = model.goal_decoder()
# pred_traj = model.pred_future_traj(, pred_goal)

UnboundLocalError: local variable 'cur_pos' referenced before assignment